In [1]:
import pandas as pd
import numpy as np
from FlagEmbedding import FlagModel
from glob import glob
from unicodedata import normalize
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

import os
from os import path
import re
import shutil

2024-09-09 18:12:44.392396: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 18:12:44.392426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 18:12:44.394560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 18:12:44.401793: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-09 18:12:45.096286: W tensorflow/compiler/tf2

In [2]:
model = FlagModel('crazyjeannot/literary_bge_base',
                  query_instruction_for_retrieval="",
                  use_fp16=True) 

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/979 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

In [3]:
maupassant = ["Quand la caissière lui eut rendu la monnaie de sa pièce de cent sous, Georges Duroy sortit du restaurant. Comme il portait beau par nature et par pose d’ancien sous-officier, il cambra sa taille, frisa sa moustache d’un geste militaire et familier, et jeta sur les dîneurs attardés un regard rapide et circulaire, un de ces regards de joli garçon, qui s’étendent comme des coups d’épervier.Les femmes avaient levé la tête vers lui, trois petites ouvrières, une maîtresse de musique entre deux âges, mal peignée, négligée, coiffée d’un chapeau toujours poussiéreux et vêtue toujours d’une robe de travers, et deux bourgeoises avec leurs maris, habituées de cette gargote à prix fixe.Lorsqu’il fut sur le trottoir, il demeura un instant immobile, se demandant ce qu’il allait faire. On était au 28 juin, et il lui restait juste en poche trois francs quarante pour finir le mois. Cela représentait deux dîners sans déjeuners, ou deux déjeuners sans dîners, au choix. Il réfléchit que les repas du matin étant de vingt-deux sous, au lieu de trente que coûtaient ceux du soir, il lui resterait, en se contentant des déjeuners, un franc vingt centimes de boni, ce qui représentait encore deux collations au pain et au saucisson, plus deux bocks sur le boulevard. C’était là sa grande dépense et son grand plaisir des nuits ; et il se mit à descendre la rue Notre-Dame-de-Lorette"]
flaubert = ["Le 15 septembre 1840, vers six heures du matin, la Ville-de-Montereau, près de partir, fumait à gros tourbillons devant le quai Saint-Bernard.Des gens arrivaient hors d’haleine ; des barriques, des câbles, des corbeilles de linge gênaient la circulation ; les matelots ne répondaient à personne ; on se heurtait ; les colis montaient entre les deux tambours, et le tapage s’absorbait dans le bruissement de la vapeur, qui, s’échappant par des plaques de tôle, enveloppait tout d’une nuée blanchâtre, tandis que la cloche, à l’avant, tintait sans discontinuer.Enfin le navire partit ; et les deux berges, peuplées de magasins, de chantiers et d’usines, filèrent comme deux larges rubans que l’on déroule.Un jeune homme de dix-huit ans, à longs cheveux et qui tenait un album sous son bras, restait auprès du gouvernail, immobile. À travers le brouillard, il contemplait des clochers, des édifices dont il ne savait pas les noms ; puis il embrassa, dans un dernier coup d’œil, l’île Saint-Louis, la Cité, Notre-Dame ; et bientôt, Paris disparaissant, il poussa un grand soupir.M. Frédéric Moreau, nouvellement reçu bachelier, s’en retournait à Nogent-sur-Seine, où il devait languir pendant deux mois, avant d’aller faire son droit. Sa mère, avec la somme indispensable, l’avait envoyé au Havre voir un oncle, dont elle espérait, pour lui, l’héritage ; il en était revenu la veille seulement ; et il se dédommageait de ne pouvoir séjourner dans la capitale, en regagnant sa province par la route la plus longue."]
anonyme = ["Il en oubliait la Maréchale, ne s’inquiétait même pas de Mme Arnoux, – ne songeant qu’à lui, à lui seul, – perdu dans les décombres de ses rêves, malade, plein de douleur et de découragement ; et, en haine du milieu factice où il avait tant souffert, il souhaita la fraîcheur de l’herbe, le repos de la province, une vie somnolente passée à l’ombre du toit natal, avec des cœurs ingénus. Le mercredi soir enfin, il sortit.Des groupes nombreux stationnaient sur le boulevard. De temps à autre, une patrouille les dissipait ; ils se reformaient derrière elle. On parlait librement, on vociférait contre la troupe des plaisanteries et des injures, sans rien de plus.– Comment ! est-ce qu’on ne va pas se battre ? dit Frédéric à un ouvrier.L’homme en blouse lui répondit :– Pas si bêtes de nous faire tuer pour les bourgeois ! Qu’ils s’arrangent !Et un monsieur grommela, tout en regardant de travers le faubourien :– Canailles de socialistes ! Si on pouvait, cette fois, les exterminer !Frédéric ne comprenait rien à tant de rancune et de sottise. Son dégoût de Paris en augmenta ; et, le surlendemain, il partit pour Nogent par le premier convoi.Les maisons bientôt disparurent, la campagne s’élargit. Seul dans son wagon et les pieds sur la banquette, il ruminait les événements des derniers jours, tout son passé. Le souvenir de Louise lui revint.« Elle m’aimait, celle-là ! J’ai eu tort de ne pas saisir ce bonheur. Bah ! n’y pensons plus !... »Puis, cinq minutes après :« Qui sait, cependant ?"]

In [4]:
proust0 = ["Et, seulement descendues dans leur jardin (peu éloigné de celui de la duchesse) pour faire la cueillette des fruits nécessaires aux compotes, venaient-elles, avant de rentrer chez elles, dire bonsoir à Mme de Guermantes chez laquelle elles n’allaient pourtant pas jusqu’à apporter un sécateur ou un arrosoir. Le duc parut touché que je fusse venu chez eux le jour même de son retour. Mais sa figure se rembrunit quand je lui eus dit que je venais demander à sa femme de s’informer si sa cousine m’avait réellement invité. Je venais d’effleurer une de ces sortes de services que M. et Mme de Guermantes n’aimaient pas rendre. Le duc me dit qu’il était trop tard, que si la princesse ne m’avait pas envoyé d’invitation, il aurait l’air d’en demander une, que déjà ses cousins lui en avaient refusé une, une fois, et qu’il ne voulait plus, ni de près, ni de loin, avoir l’air de se mêler de leurs listes, « de s’immiscer », enfin qu’il ne savait même pas si lui et sa femme, qui dînaient en ville, ne rentreraient pas aussitôt après chez eux, que dans ce cas leur meilleure excuse de n’être pas allés à la soirée de la princesse était de lui cacher leur retour à Paris, que, certainement sans cela, ils se seraient au contraire empressés de lui faire connaître en lui envoyant un mot ou un coup de téléphone à mon sujet, et certainement trop tard, car en toute hypothèse les listes de la princesse étaient certainement closes. « Vous n’êtes pas mal avec elle », me dit-il d’un air soupçonneux, les Guermantes craignant toujours de ne pas être au courant des dernières brouilles et qu’on ne cherchât à se raccommoder sur leur dos. Enfin comme le duc avait l’habitude de prendre sur lui toutes les décisions qui pouvaient sembler peu aimables : « Tenez, mon petit, me dit-il tout à coup, comme si l’idée lui en venait brusquement à l’esprit, j’ai même envie de ne pas dire du tout à Oriane que vous m’avez parlé de cela. Vous savez comme elle est aimable, de plus elle vous aime énormément, elle voudrait envoyer chez sa cousine malgré tout ce que je pourrais lui dire, et si elle est fatiguée après dîner, il n’y aura plus d’excuse, elle sera forcée d’aller à la soirée. Non, décidément, je ne lui en dirai rien. Du reste vous allez la voir tout à l’heure. Pas un mot de cela, je vous prie. Si vous vous décidez à aller à la soirée je n’ai pas besoin de vous dire quelle joie nous aurons de passer la soirée avec vous. »"]
proust1 = ["Mais ce cas était le plus rare ; d’ailleurs, quand le jour de la présentation arrivait enfin (à un moment où elle était d’ordinaire déjà assez indifférente au duc, dont les actions, comme celles de tout le monde, étaient plus souvent commandées par les actions antérieures, dont le mobile premier n’existait plus) il se trouvait souvent que ç’avait été Mme de Guermantes qui avait cherché à recevoir la maîtresse en qui elle espérait et avait si grand besoin de rencontrer, contre son terrible époux, une précieuse alliée. Ce n’est pas que, sauf à de rares moments, chez lui, où, quand la duchesse parlait trop, il laissait échapper des paroles et surtout des silences qui foudroyaient, M. de Guermantes manquât vis-à-vis de sa femme de ce qu’on appelle les formes. Les gens qui ne les connaissaient pas pouvaient s’y tromper. Quelquefois, à l’automne, entre les courses de Deauville, les eaux et le départ pour Guermantes et les chasses, dans les quelques semaines qu’on passe à Paris, comme la duchesse aimait le café-concert, le duc allait avec elle y passer une soirée. Le public remarquait tout de suite, dans une de ces petites baignoires découvertes où l’on ne tient que deux, cet Hercule en « smoking » (puisqu’en France on donne à toute chose plus ou moins britannique le nom qu’elle ne porte pas en Angleterre), le monocle à l’œil, dans sa grosse mais belle main, à l’annulaire de laquelle brillait un saphir, un gros cigare dont il tirait de temps à autre une bouffée, les regards habituellement tournés vers la scène, mais, quand il les laissait tomber sur le parterre où il ne connaissait d’ailleurs absolument personne, les émoussant d’un air de douceur, de réserve, de politesse, de considération. Quand un couplet lui semblait drôle et pas trop indécent, le duc se retournait en souriant vers sa femme, partageait avec elle, d’un signe d’intelligence et de bonté, l’innocente gaieté que lui procurait la chanson nouvelle. Et les spectateurs pouvaient croire qu’il n’était pas de meilleur mari que lui ni de personne plus enviable que la duchesse — cette femme en dehors de laquelle étaient pour le duc tous les intérêts de la vie, cette femme qu’il n’aimait pas, qu’il n’avait jamais cessé de tromper ; — quand la duchesse se sentait fatiguée, ils voyaient M. de Guermantes se lever, lui passer lui-même son manteau en arrangeant ses colliers pour qu’ils ne se prissent pas dans la doublure, et lui frayer un chemin jusqu’à la sortie avec des soins empressés et respectueux qu’elle recevait avec la froideur de la mondaine qui ne voit là que du simple savoir-vivre, et parfois même avec l’amertume un peu ironique de l’épouse désabusée qui n’a plus aucune illusion à perdre."]

In [5]:
chateau = ["Lorsqu’en 1806 j’entrepris le voyage d’outre-mer, Jérusalem était presque oubliée ; un siècle anti-religieux avait perdu mémoire du berceau de la religion : comme il n’y avait plus de chevaliers, il semblait qu’il n’y eût plus de Palestine.  Le dernier voyageur dans le Levant, M. le comte de Volney, avait donné au public d’excellents renseignements sur la Syrie ; mais il s’était borné à des détails généraux sur le gouvernement de la Judée."]

In [6]:
mohican_dialogue = [" — Comment ! il est parti pour Rome ? s’écria M. Gérard ; l’abbé Dominique est parti pour Rome ?  — Eh ! mon Dieu, oui !  — Et qu’est-il allé faire à Rome ?  — Mon cher monsieur Gérard, il n’y a qu’un homme qui puisse relever l’abbé Dominique du secret de la confession.  — Oui, le pape.  — Eh bien, il est allé demander au pape de le relever de ce secret.  — Oh ! mon Dieu !  — C’est pour avoir le temps de faire le voyage qu’il a sollicité et obtenu du roi un sursis.  — Mais je suis perdu, alors ! s’écria M. Gérard.  — Pourquoi cela ?  — Le pape lui accordera sa demande.  M. Jackal secoua la tête.  — Non, vous croyez que non ?  — J’en suis sûr, monsieur Gérard.  — Comment en êtes-vous sûr ?  — Je connais Sa Sainteté.  — Vous avez l’honneur de connaître le pape ?  — Comme la police a l’honneur de tout connaître, monsieur Gérard ; "]
mohican_récit = ["C’était le lendemain ou le surlendemain de la visite de M. Jackal à M. Gérard – car on comprendra qu’il nous est impossible, à un jour près, de renseigner positivement nos lecteurs : nous suivons l’ordre chronologique des événements, voilà tout. – Il était dix heures et demie du matin. Pétrus, Ludovic et Jean Robert étaient assis : Pétrus dans une bergère, Ludovic sur un fauteuil Rubens, Jean Robert dans un immense voltaire. Chacun d’eux avait à la portée de sa main une tasse de thé plus ou moins vide, et, dans le milieu de l’atelier, une table encore servie indiquait que le thé était employé, comme digestif, à la suite d’un déjeuner substantiel.  Un manuscrit écrit en lignes inégales – en vers par conséquent –, dont les cinq actes séparés gisaient confusément à terre, à la droite de Jean Robert, prouvait que le poète venait de faire une lecture"]

In [7]:
SA = [" Nous signons. Pinaud qui fait office de maire continue de fourbir sa braguette éclaboussée d’encre en marmonnant des consternations. Il ronchonne comme quoi faut être gestapiste dans l’âme pour placer un encrier boiteux sur la table des mariages à une époque où la pointe Bic submerge la France.  Avouez qu’il a raison, Pinuchet. Le régisseur de la maison Poupoule qui nous a organisé cette mairie-bidon à Chaulx-lez-Maron (Yvelines) a trop forcé sur le folklore. Il les a vues seulement dans les films de Berthomieu, les petites mairies de nos chères campagnes. Y aurait pas la binette à Carolus en gévacolor sur le mur, on se croirait dans un vaudeville datant du muet. Heureusement que notre Colon-les-deux-étoiles est là pour affirmer son époque (A l’heure où nous mettons sous presse, les photos du président de Pompidou ne sont pas encore sèches à l’instar des chemises de l’archiduchesse).  — Au lieu de rouscailler, m’sieur le maire de ma paire ferait p’t’être mieux de gazouiller un laïus aux jeunes z’époux ! déclare le Dodu."]

In [8]:
Nothomb = ["Soudain, il se souvint d’un conte d’Oscar Wilde qui racontait une histoire similaire. La bibliothèque du Pluvier était si désordonnée qu’y retrouver un livre relevait du miracle.  Neville préféra se rendre à la librairie du village. Sur le catalogue Folio, il repéra le titre d’Oscar Wilde : Le Crime de lord Arthur Savile. Le libraire en possédait un exemplaire. De retour chez lui, Henri s’isola avec le livre qu’il dévora. Dans sa jeunesse, il l’avait lu avec hilarité : il comprenait à présent la gravité de cette affaire.  Sur le point d’épouser la belle Sybil dont il était éperdument amoureux, lord Arthur Savile lors d’une soirée à Londres se fit lire les lignes de la main par un chiromancien renommé qui lui annonça qu’il allait commettre un crime. En proie au désespoir, lord Arthur erra toute la nuit avant d’ajourner son mariage. Il lui fallait se débarrasser de la sale besogne avant d’unir son destin avec celle qu’il aimait. On ne racontera pas ici les péripéties de ce noble anglais, pris entre les exigences contraires du devoir, de l’étiquette et de l’amour afin de préserver le plaisir de lecture des intéressés que l’on espère nombreux."]

In [37]:
embeddings_1 = model.encode(mohican_dialogue)
embeddings_2 = model.encode(mohican_récit)
embeddings_3 = model.encode(proust1)

In [38]:
embeddings = [embeddings_1[0], embeddings_2[0], embeddings_3[0]]

In [39]:
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)

In [40]:
cosine_similarity(scaled_embeddings)

array([[ 1.        , -0.45625247, -0.49331458],
       [-0.45625247,  1.        , -0.54896122],
       [-0.49331458, -0.54896122,  1.        ]])